In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn import linear_model as ln

data=pd.read_csv('../input/train.csv')

In [ ]:
data.head()

## EXPLORATORY DATA ANALYSIS

In [ ]:
data.describe()

In [ ]:
data['SalePrice'].describe()

In [ ]:
#histogram
sns.distplot(data['SalePrice']);

In [ ]:
sns.boxplot(data['SalePrice'])
plt.show()
print ("mean = %lf",data['SalePrice'].mean())
print ("std = %lf" ,data['SalePrice'].std())
print ("median = %lf" ,data['SalePrice'].median())
print("Skewness: %f" , data['SalePrice'].skew())
print("Kurtosis: %f" , data['SalePrice'].kurt())

Moreover, it is clear that we could have to deal with a certain number of outliers. Let us firsrt looking for potential relationships between the different numerical/categorical inputs and the output target variable (SalePrice).

### Numerical features

Let us start analyzing numerical variables.

In [ ]:
# This command allows you to easily determine the numerical variables
# In this way, we can easily produces a series of scatter plots, useful for visually detecting some potential correlation
numerical=data._get_numeric_data().columns
numerical=numerical.drop('SalePrice')
numerical=numerical.drop('Id')
In [8]:


In [ ]:
target='SalePrice'
Y=data[target].values
for x in numerical:
    X=data[x].values
    plt.xlabel(x)
    plt.ylabel(target)
    plt.scatter(X,Y)
    plt.show()

Some of the features show a quite clear linear relation with the target variable 'SalePrice', while others not, due to the amount of noisyness that characterize them. Moreover it seems that some of this input features contain a quite small number of observations (i.e. many missing values), and then could be of no use for our regression study.

### Categorical features

Now, let us have a look at categorical features: seaborn allow us to easily produce bar charts plots that could reveal some interesting relation

In [ ]:
categorical=data.columns.drop(numerical).drop(target).drop('Id')
for i in categorical:
    
    g=sns.boxplot(x=i, y=target, data=data[[i,target]]);
    for item in g.get_xticklabels():
        item.set_rotation(60)
    plt.show()

There could be some interesting relation between our target variable and several categorical and numerical features.

Before applying a regression model, it is better to look for some potential collinearity among our numerical features: this can be easily done computing a matrix of correlation coefficients.

In [ ]:
#saleprice correlation matrix
fig, ax = plt.subplots(figsize=(10,10))
corrmat = data.corr()#(method='spearman')
k = 20 #number of variables for heatmap
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index # takes the first k entries with largest correlation
cm = np.corrcoef(data[cols].values.T)
sns.set(font_scale=1.15)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

Hence, there it seems to be some collinearity between our input variables: this tells us that we will have to use regression regularized regression algorithms in order to have a properly performing learner.

In [ ]:
# keeping only features which show (even if weakly) a correlation 
# with our target variable. I am not sure how much solid this approach could be.

threshold=.3 
rel=[]
relevant=[]
for x in cols.drop(target):
    coeff=np.corrcoef(data[[x,target]].values.T)[1,0]
    if coeff > threshold:
         rel.append([x,coeff])
         relevant.append(x)   
rel.sort(key=lambda x: x[1])
relevant

#### Let us just give a look at missing data

In [ ]:
#missing data
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

A good way of proceeding is to eliminate those features whose missing percentage is more than 15%. For the remaining features, missing values are filled using substitution by imputation.

In [ ]:
tr=.15
for j in missing_data.index.values:
     if missing_data['Percent'].loc[j]>tr:
            data=data.drop(j, 1)
            if j in relevant:
                relevant.remove(j)
relevant

#### Updating the relevant categorical variables

In [ ]:
numerical=data._get_numeric_data().columns
categorical=data.columns.drop(numerical)

### FEATURES ENGINEERING
After having collected all the potentially interesting features, we need to subsitute missing values and apply log transformations on both target and input variables, followed by a standardization of the latter (this should enhance the convergence of our training algorithms). The dummy variables need to be created.

In [ ]:
from scipy.stats import norm
#applying log transformation on target variable
train=data
train['SalePrice'] = np.log(train['SalePrice'])

#transformed histogram and normal probability plot
sns.distplot(train['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(train['SalePrice'], plot=plt)

Log transformation on target variable works pretty well: the skewness has been quite reduced, and it seems we should not worry about outliers(for the moment).

Now, let's have a look at our numerical input features, fill their missing values, and see if some of them could benefit from the application of a log-transformation

In [ ]:
from sklearn.preprocessing import Imputer
from scipy.stats import norm

imput=Imputer(missing_values='NaN', strategy='mean')
imput.fit_transform(train[relevant])
train[relevant]=imput.transform(train[relevant])

Y=data[target].values
for x in relevant:
    if train[x].min()>0:
        X=np.log(train[x]).values
        print train[x].describe()
        plt.xlabel(x)
        plt.ylabel(target)
        plt.scatter(X,Y)
        plt.show()

We can take the log-transformation only for those features whose minimum value is not zero. From the plots above it seems that log-transforming candidate numerical features could help in having enough homoscedasticity, which is one of the fundamental requirements for applying linear regression models.

In [ ]:
to_log=[]
for x in relevant:
    if train[x].min()!=0:
        to_log.append(x)

In [ ]:
train[to_log]=np.log(train[to_log])

Now, we apply standardization, in order to have predictors centered around zero, with a common range scale

In [ ]:
from sklearn import preprocessing as ppr


stand=ppr.StandardScaler()
train[relevant]=stand.fit_transform(train[relevant])

Now it's time to add dummy variables: at this point it quit important to first investigate the shapes of train and test set, in order to understand which feature can be retained and which not

In [ ]:
test=pd.read_csv('../input/test.csv')
test[relevant].head()

In [ ]:
new_catg=[]
for i in categorical:
    dummy1=pd.get_dummies(train[i]).shape[1]
    dummy2=pd.get_dummies(test[i]).shape[1]
    if dummy1==dummy2:
        new_catg.append(i)


The names included in "new_catg" represents categories which mantain the same cardinality from train to test set, hence can be used for prediction. Finally, we create our final train set:

In [ ]:
X=pd.concat([train[relevant],pd.get_dummies(train[new_catg])],axis=1)

Before proceeding with the training of the learners, we already transform the test set

In [ ]:
from sklearn.preprocessing import Imputer

imput=Imputer(missing_values='NaN', strategy='mean')
imput.fit_transform(test[relevant])
test[relevant]=imput.transform(test[relevant])

Now apply log transformation and standardization

In [ ]:
test[to_log]=np.log(test[to_log])

In [ ]:
from sklearn import preprocessing as ppr

stand=ppr.StandardScaler()
test[relevant]=stand.fit_transform(test[relevant])

In [ ]:
Xtest=pd.concat([test[relevant],pd.get_dummies(test[new_catg])],axis=1)

Now training and test set are ready, and we can focus on the implementation of learning algorithms.


#### MODEL SELECTION AND OPTIMIZATION
Here we are! This is the most (in my opinion) exciting part of a data science project: model selection and optimization. Here we focus on different types of regression learners that could be used for predicting the SalePrice variable. Each learner comes with its specific strength and weak point, and a certain number of hyper parameters that need to be trained in order to

In [ ]:
keys=["alpha","intercept","coefficients"]


import time

### Lasso: parameter tuning¶

In [ ]:
LASSOdic=dict.fromkeys(keys,None)
y=train[target]
print("Computing regularization path using the coordinate descent lasso...")
t1 = time.time()
model = ln.LassoCV(cv=5).fit(X, y)
t_lasso_cv = time.time() - t1

# Display results
m_log_alphas = -np.log10(model.alphas_)

plt.figure()
#ymin, ymax = 2300, 3800
plt.plot(m_log_alphas, model.mse_path_, ':')
plt.plot(m_log_alphas, model.mse_path_.mean(axis=-1), 'k',
         label='Average across the folds', linewidth=2)
plt.axvline(-np.log10(model.alpha_), linestyle='--', color='k',
            label='alpha: CV estimate')

plt.legend()

plt.xlabel('-log(alpha)')
plt.ylabel('Mean square error')
plt.title('Mean square error on each fold: coordinate descent '
          '(train time: %.2fs)' % t_lasso_cv)
plt.axis('tight')

LASSOdic["coefficients"]=model.coef_
LASSOdic["alpha"]=model.alpha_
LASSOdic["intercept"]=model.intercept_

LASSOdic

In [ ]:
clfLA=ln.Lasso(alpha=LASSOdic["alpha"])
clfLA.fit(X,y)
clfLA.score(X,y)

### Ridge: parameter tuning

In [ ]:
RIDGEdic=dict.fromkeys(keys,None)
print("Computing regularization path using the coordinate descent Ridge...")
t1 = time.time()
model = ln.RidgeCV(cv=5).fit(X, y)
t_ridge_cv = time.time() - t1

RIDGEdic["coefficients"]=model.coef_
RIDGEdic["alpha"]=model.alpha_
RIDGEdic["intercept"]=model.intercept_

RIDGEdic

In [ ]:
clfRI=ln.Ridge(alpha=RIDGEdic["alpha"])
clfRI.fit(X,y)
clfRI.score(X,y)

In [ ]:
### ElasticNet: parameter tuning

In [ ]:
keys=["alpha","l1ratio","intercept","coefficients"]
ELNETdic=dict.fromkeys(keys,None)
print("Computing regularization path using the coordinate descent elasticnet...")
t1 = time.time()
model = ln.ElasticNetCV(cv=5).fit(X, y)
t_lasso_cv = time.time() - t1

# Display results
m_log_alphas = -np.log10(model.alphas_)

plt.figure()
#ymin, ymax = 2300, 3800
plt.plot(m_log_alphas, model.mse_path_, ':')
plt.plot(m_log_alphas, model.mse_path_.mean(axis=-1), 'k',
         label='Average across the folds', linewidth=2)
plt.axvline(-np.log10(model.alpha_), linestyle='--', color='k',
            label='alpha: CV estimate')

plt.legend()

plt.xlabel('-log(alpha)')
plt.ylabel('Mean square error')
plt.title('Mean square error on each fold: coordinate descent '
          '(train time: %.2fs)' % t_lasso_cv)
plt.axis('tight')
#plt.ylim(ymin, ymax)

ELNETdic["alpha"]=model.alpha_
ELNETdic["coefficients"]=model.coef_
ELNETdic["l1ratio"]=model.l1_ratio_
ELNETdic["intercept"]=model.intercept_

ELNETdic

In [ ]:
clfEL=ln.ElasticNet(alpha=ELNETdic["alpha"],l1_ratio=ELNETdic["l1ratio"])
clfEL.fit(X,y)
clfEL.score(X,y)

### KNN: tuning number of neighbors

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
import time

scoresk = list()
y=train[target]
n_folds =5
to_plot=[]
times=[]
for i in range(1,30):
    t1 = time.time()
    clf = KNeighborsRegressor(n_neighbors=i,weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski')
    this_scores = cross_val_score(clf, X, y, cv=n_folds, n_jobs=-1)
    t_train=time.time() - t1
    times.append(t_train)
    scoresk.append([i,np.mean(this_scores),np.std(this_scores)])
    to_plot.append(list(this_scores))

optk=max(scoresk,key=lambda x: x[1])

### Plotting our tuning procedure